a.k.a full_weekdaycalls <br />
Author: Joshua, Will, Ethan <br />
Summary: Collect calls and group them into calls by hour


In [9]:
import csv
import collections
import pandas as pd
import numpy as np
import math
import json

In [10]:
# This is the grid object, which is used throughout all data preprocessing.
# It represents the city of Austin through a series of grids.
# It thus makes a tractable way to compute distance between grids, ect.
class Grid():
    def __init__(self, grid_json):
        self.grid = grid_json
        self.min_lat = self.grid["latitude_min"]
        self.min_lon = self.grid["longitude_min"]
        self.max_lat = self.grid["latitude_max"]
        self.max_lon = self.grid["longitude_max"]
        self.latitude_delta = self.grid["latitude_step"]
        self.longitude_delta = self.grid["longitude_step"]
        self.nrows = math.ceil((self.max_lat - self.min_lat) / self.latitude_delta)
        self.ncols = math.ceil((self.max_lon - self.min_lon) / self.longitude_delta)
        self.times = self.grid["time_matrix"]
        self.census_tract_region_map = self.grid["census_tract_region_mapping"]
        self.region_to_tract = collections.defaultdict(list)
        for census_tract in self.census_tract_region_map:
            for region in self.census_tract_region_map[census_tract]:
                self.region_to_tract[region].append(census_tract)
    def map_point_to_region(self, latitude, longitude):
        return math.floor((latitude-self.min_lat)/self.latitude_delta) * self.ncols  + math.floor((longitude-self.min_lon)/self.longitude_delta)
    def get_representative(self, region_num):
        row_num = region_num//self.ncols
        col_num = region_num - row_num*self.ncols
        lat = self.min_lat + row_num * self.latitude_delta + 0.5*self.latitude_delta
        lon = self.min_lon + col_num * self.longitude_delta + 0.5*self.longitude_delta
        return [lon, lat]
    def get_time(self, region1, region2):
        try:
            return self.times[region1][region2]
        except IndexError:
            return -1
    def region_to_census_tract(self, region):
        try:
            return self.region_to_tract[region]
        except KeyError:
            return "0_0"

In [11]:
with open("Input_data/cleaned_data.csv", "r") as f:
    data = pd.read_csv(f, error_bad_lines=False)
orig_data_size = len(data)

## Collect calls and group them into calls by hour

In [12]:
with open("Input_data/grid_info_smaller.json", "r") as f:
    grid_json = json.load(f)

In [13]:
g = Grid(grid_json)

In [14]:
#filter?
data["region_num"] = data.apply(lambda x: g.map_point_to_region(x["Latitude_Of_Emergency"], x["Longitude_Of_Emergency"]), axis=1)

In [15]:
data

,Unnamed: 0,IncidentForeignKey,Radio_Name,Longitude_At_Assign_Time,Latitude_At_Assign_Time,Time_Assigned,Time_Enroute,Time_ArrivedAtScene,Time_Depart_Scene,Time_Arrive_Destination,Time_Available,Time_Call_Cleared,Call_Disposition,Longitude_Of_Emergency,Latitude_Of_Emergency,transport_time,grid_time,region_num
0,1,23397873,M09,-97.972866,30.333385,2019-01-01 19:43:36.000,2019-01-01 19:45:27.030,2019-01-01 19:48:39.770,2019-01-01 20:19:00.870,2019-01-01 20:24:43.093,2019-01-01 20:51:12.877,2019-01-01 20:51:12.877,Baylor Scott & White - Lakeway,-97.961,30.325,192,222.72,93
1,3,23408585,DM03,-97.751235,30.246114,2019-01-03 14:20:41.000,2019-01-03 14:20:54.110,2019-01-03 14:27:27.613,NaN,NaN,2019-01-03 14:35:14.657,2019-01-03 14:35:14.657,False Alarm Call,-97.777,30.251,393,343.52,82
2,4,23408585,DMO02,-97.751966,30.245411,2019-01-03 14:20:58.117,2019-01-03 14:20:58.140,2019-01-03 14:26:59.083,NaN,NaN,2019-01-03 14:34:24.343,2019-01-03 14:34:24.343,Dual w/ other ATCEMS Unit,-97.777,30.251,360,343.52,82
3,8,23439455,M10,-97.742590,30.418169,2019-01-07 17:28:48.000,2019-01-07 17:30:12.067,2019-01-07 17:47:53.030,NaN,NaN,2019-01-07 17:56:40.097,2019-01-07 17:56:40.097,False Alarm Call,-97.747,30.407,1060,412.58,127
4,10,23444785,M04,-97.725892,30.290517,2019-01-08 13:55:38.000,2019-01-08 13:55:55.987,2019-01-08 14:04:26.413,2019-01-08 14:20:11.810,2019-01-08 14:41:12.577,2019-01-08 15:09:21.000,2019-01-08 15:09:21.000,Saint Davids Med Ctr,-97.626,30.294,510,997.13,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210102,279971,27262452,CHP11,-97.769524,30.231354,2020-06-29 14:57:29.090,2020-06-29 14:57:29.090,2020-06-29 15:33:16.447,NaN,NaN,2020-06-29 16:50:48.823,2020-06-29 16:50:48.823,Other,-97.654,30.379,2147,1318.73,129
210103,279972,27317236,CHP11,-97.774368,30.224984,2020-07-07 10:03:20.000,2020-07-07 10:03:55.657,2020-07-07 11:17:14.023,NaN,NaN,2020-07-07 12:10:23.387,2020-07-07 12:10:23.387,Other,-97.789,30.204,4398,304.24,66
210104,279973,27336505,CHP21,-97.734738,30.275694,2020-07-10 09:10:57.000,2020-07-10 09:11:46.533,2020-07-10 09:22:19.883,NaN,NaN,2020-07-10 09:36:30.773,2020-07-10 09:36:30.773,Other,-97.767,30.190,633,754.85,52
210105,279974,27413735,CHP08,-97.770123,30.191624,2020-07-22 13:10:46.880,2020-07-22 13:10:46.880,2020-07-22 13:42:29.970,NaN,NaN,2020-07-22 14:13:23.757,2020-07-22 14:13:23.757,CHP Intervention(CHP use only),-97.694,30.227,1903,757.36,68


In [16]:
from datetime import datetime, timedelta

In [17]:
data["year"] = data.apply(lambda x: datetime.strptime(x["Time_Assigned"], "%Y-%m-%d %H:%M:%S.%f").year, axis=1)

In [18]:
data["month"] = data.apply(lambda x: datetime.strptime(x["Time_Assigned"], "%Y-%m-%d %H:%M:%S.%f").month, axis=1)

In [19]:
data["day"] = data.apply(lambda x: datetime.strptime(x["Time_Assigned"], "%Y-%m-%d %H:%M:%S.%f").day, axis=1)

In [20]:
data["hour"] = data.apply(lambda x: datetime.strptime(x["Time_Assigned"], "%Y-%m-%d %H:%M:%S.%f").hour, axis=1)

In [21]:
data["S"] = 1

In [22]:
d = data.groupby(["year", "month", "day", "hour", "region_num"])["S"].sum()

In [23]:
d.keys()

MultiIndex([(2019, 1,  1, 0,  36),
            (2019, 1,  1, 0,  51),
            (2019, 1,  1, 0,  54),
            (2019, 1,  1, 0,  64),
            (2019, 1,  1, 0,  67),
            (2019, 1,  1, 0,  82),
            (2019, 1,  1, 0,  83),
            (2019, 1,  1, 0,  98),
            (2019, 1,  1, 0, 113),
            (2019, 1,  1, 0, 127),
            ...
            (2020, 8, 24, 8,  83),
            (2020, 8, 24, 8,  98),
            (2020, 8, 24, 8, 110),
            (2020, 8, 24, 8, 112),
            (2020, 8, 24, 8, 113),
            (2020, 8, 24, 8, 142),
            (2020, 8, 24, 9,  51),
            (2020, 8, 24, 9,  68),
            (2020, 8, 24, 9,  98),
            (2020, 8, 24, 9,  99)],
           names=['year', 'month', 'day', 'hour', 'region_num'], length=143088)

In [24]:
#p is the final object that we will be exporting to a .csv
p = pd.DataFrame(columns=["year", "month", "day", "hour"]+[i+1 for i in range(210)]) #indexes are off by 1

In [25]:
p

,year,month,day,hour,1,2,3,4,5,6,...,201,202,203,204,205,206,207,208,209,210


In [26]:
d.keys()[0][0]

2019

In [27]:
#puts the entrues from d into p
year = d.keys()[0][0] #keys ->
month = d.keys()[0][1]
day = d.keys()[0][2]
hour = d.keys()[0][3]
curr = datetime(year=year, month=month, day=day, hour=hour)
t = timedelta(hours=1)
years = [] #-> new data
months = []
days = []
hours = []
while not (curr.year == d.keys()[-1][0] and curr.month == d.keys()[-1][1] and curr.day == d.keys()[-1][2]):
    if curr.weekday() < 5:
        years.append(curr.year)
        months.append(curr.month)
        days.append(curr.day)
        hours.append(curr.hour)
    curr += t

p["year"] = years
p["month"] = months
p["day"] = days
p["hour"] = hours

In [29]:
for i in range(1, g.nrows*g.ncols+1):
    p[i] = 0

In [30]:
#for i in range(3201, ):
    #p[i] = 0
p

,year,month,day,hour,1,2,3,4,5,6,...,201,202,203,204,205,206,207,208,209,210
0,2019,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2019,1,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019,1,1,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2019,1,1,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10291,2020,8,21,19,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10292,2020,8,21,20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10293,2020,8,21,21,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10294,2020,8,21,22,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
for k in d.keys():
    p.loc[(p["year"] == k[0]) & (p["month"] == k[1]) & (p["day"] == k[2]) & (p["hour"] == k[3]), k[4]] = d[k]

In [32]:
p.columns

Index([ 'year', 'month',   'day',  'hour',       1,       2,       3,       4,
             5,       6,
       ...
           -22,      -5,     239,     262,    -200,      -6,     231,    -234,
           220,     -10],
      dtype='object', length=226)

In [33]:
#remove collumn
for col in p.columns[228:]:
    p = p.drop(col, axis=1)

In [34]:
p

,year,month,day,hour,1,2,3,4,5,6,...,-22,-5,239,262,-200,-6,231,-234,220,-10
0,2019,1,1,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,1,1,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,1,1,2,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,1,1,3,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,1,1,4,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10291,2020,8,21,19,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10292,2020,8,21,20,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10293,2020,8,21,21,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10294,2020,8,21,22,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
p.to_csv("Output_Data/Hourly_Calls.csv")

In [37]:
p_n = p.to_numpy()

In [46]:
sum(p_n[1,3:200])

38.0